In [1]:
import os
from src.utils import read_config, MySQLAgent
import openai
from llama_index.llms.openai import OpenAI

from llama_index.core import SQLDatabase, Settings
from llama_index.core.query_engine import NLSQLTableQueryEngine, RouterQueryEngine
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import QueryEngineTool
from llama_index.core.selectors import LLMSingleSelector

from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column
)

c:\Users\sean.chang\AppData\Local\anaconda3\envs\sean_project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
info_path = '.env/info.json'
info = read_config(info_path)

O_KEY = info['okey']
os.environ["OPENAI_API_KEY"] = O_KEY
openai.api_key = os.environ["OPENAI_API_KEY"]

conn_info = info['VM1_news_mysql_conn_info']
sql_agent = MySQLAgent(conn_info)

In [3]:
llm =  OpenAI(temperature=0.1, model="gpt-4o-mini")

In [4]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embed_model = HuggingFaceEmbedding(model_name=model_name)

In [5]:
user = conn_info['user']
pw = conn_info['pw']
host = conn_info['host']
port = conn_info['port']
database = conn_info['database']
connection_string = f"mysql+pymysql://{user}:{pw}@{host}:{port}/{database}?charset=utf8mb4"
engine = create_engine(connection_string)

In [6]:
tables = ['cdd_result']

sql_database = SQLDatabase(engine, include_tables=tables)

Settings.llm = llm
Settings.embed_model = embed_model
sql_query_engine = NLSQLTableQueryEngine(sql_database=sql_database, tables=tables)

In [7]:
prompt_template_str = """
###情況判別
如果Query中出現詢問綠燈紅燈黃燈的話則搜尋light_status這個欄位
"""

In [8]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=('當User詢問Cdd result的時候搜尋這個Table')
)

In [9]:
query_engine =  RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(prompt_template_str=prompt_template_str),
    query_engine_tools=[sql_tool]
)

In [10]:
query = """旭全發企業有限公司的有幾筆light_status是green的資料"""
response = query_engine.query(query)
response.response

'旭全發企業有限公司的light_status為green的資料共有126筆。'

In [14]:
metadata = response.metadata
first_key = next(iter(metadata))
metadata[first_key]['sql_query']

"SELECT COUNT(*) AS green_count FROM cdd_result WHERE company_name = '旭全發企業有限公司' AND light_status = 'green'"